In [1]:
import pandas as pd # דאטא ובטבלאות
import numpy as np # ספריה חישובית
import matplotlib.pyplot as plt # הצגת גרפים
from pylab import rcParams # שינוי גודל הגרפים
from pylab import * 
from IPython.display import Image # הצגת תמונות

#from matplotlib import pyplot  
import statsmodels.api as sm # חישוב וחלוקה לטרנד, מחזוריות ושאריות

from pmdarima import model_selection, auto_arima # Arima model & Train/Test split for Time series (and not random)
import pmdarima as pm # Arima Library
from prophet import Prophet #Prophet model
from sklearn.ensemble import RandomForestRegressor # Random Forest model

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error # מדדי השוואה


In [7]:
df_all = pd.read_csv("C:/Users/yonad/OneDrive - Bar-Ilan University/Corona Final Project/Data/City_Status_Vaccine_Total.csv")

In [56]:
df_all.City_Code.unique()

array([  26,   28,   31,   41,   46,   47,   53,   70,   99,  122,  147,
        154,  166,  168,  171,  182,  195,  229,  240,  246,  276,  326,
        351,  466,  469,  472,  473,  478,  480,  481,  482,  483,  485,
        487,  489,  490,  492,  494,  496,  498,  499,  502,  504,  505,
        507,  508,  509,  510,  511,  512,  516,  517,  518,  520,  521,
        522,  524,  525,  526,  528,  529,  530,  531,  532,  534,  535,
        536,  537,  538,  541,  542,  543,  565,  577,  587,  627,  628,
        633,  634,  635,  637,  638,  652,  654,  658,  666,  681,  696,
        709,  763,  812,  831,  874,  913,  922,  944,  962,  975,  978,
        998, 1015, 1020, 1031, 1034, 1050, 1054, 1059, 1060, 1061, 1063,
       1066, 1082, 1098, 1113, 1137, 1139, 1161, 1167, 1186, 1192, 1200,
       1224, 1243, 1245, 1247, 1263, 1268, 1271, 1274, 1284, 1286, 1290,
       1292, 1295, 1296, 1303, 1304, 1309, 1310, 1315, 1319, 1326, 1327,
       1330, 1333, 1337, 1345, 1347, 1348, 1358, 13

In [ ]:
future_val_all = pd.DataFrame(columns=['val', 'city', 'day'])
for City_Code in df_all.City_Code.unique():
    print(City_Code)
    df = df_all[df_all['City_Code'] == City_Code][["Date","Accumulated_Verified_Cases"]].copy()
    df['Date'] = pd.to_datetime(df['Date'],format='%d/%m/%Y')

    df_arima = df.set_index('Date')
    df_arima["avc_diff"] = df_arima.Accumulated_Verified_Cases.diff()
    df_arima.dropna(inplace=True) # מוחק רעשים 
    
    train, test = model_selection.train_test_split(df_arima['avc_diff'],train_size = 0.8)
    arima = pm.auto_arima(train,seasonal=True)
    forecast = arima.predict(n_periods=len(test)) 
    arima_rmse = np.sqrt(mean_squared_error(test, forecast))

    df_prophet = df_arima.copy()
    df_prophet.reset_index(inplace=True)
    df_prophet.rename(columns={'Date': 'ds','avc_diff': 'y'}, inplace=True)    

    train, test = model_selection.train_test_split(df_prophet,train_size = 0.8)
    try:
        m = Prophet() 
        m.fit(train)

        future = m.make_future_dataframe(periods=len(test)) 
        forecast = m.predict(future)
        prophet_rmse = np.sqrt(mean_squared_error(test.y, forecast.yhat[-len(test):])) 
    except:
        prophet_rmse = 10000000 #to make sure arima will be selected.
    
    if (arima_rmse>prophet_rmse):
        try:
            m.fit(df_prophet) 
            future = m.make_future_dataframe(periods=14) 
            forecast = m.predict(future)

            future_val = forecast.yhat[-14:].to_list()
        except:
            arima = pm.auto_arima(df_arima['avc_diff']) 
            future_val = list(arima.predict(n_periods=14))
    else:
        arima = pm.auto_arima(df_arima['avc_diff']) 
        future_val = list(arima.predict(n_periods=14))
 

    for i in range(0, len(future_val)):
        future_val[i] = int(future_val[i])
    
    temp = pd.DataFrame(future_val, columns=['val'])
    temp['city'] = City_Code
    temp['day'] = temp.index
    
    future_val_all = pd.concat([future_val_all,temp])

future_val_all.reset_index().to_csv('prediction.csv')

26


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


28


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


31


...

3778


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


3779


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


3780


In [55]:
future_val_all.reset_index().to_csv('prediction.csv')